Lecture 13 will cover further go into examples on how to tie the GUI-widgets to the application logic by looking into more on the signal handling in Qt.

First a bit of standard stuff needed to work with Qt

In [1]:
#from PySide.QtCore import *
#from PySide.QtGui import *
#from PyQt4.QtCore import *
#from PyQt4.QtGui import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
import sys
qt_app = QApplication(sys.argv)

# Signals

Lets start by creating a simpel model that emits a signal when the data has been modified.

In [18]:
class MyCounterModel(QObject):
    # We create a new signal
    data_changed = pyqtSignal()
    
    def __init__(self):
        super().__init__()
        self.count = 0

    def value(self):
        return self.count

    def increment(self):
        self.count += 1
        self.data_changed.emit()

In [6]:
class MyCounterView(QWidget):
    def __init__(self, model):
        super().__init__()
        self.button = QPushButton("Add 1")
        self.label = QLabel()

        layout = QHBoxLayout()
        layout.addWidget(self.button)
        layout.addWidget(self.label)
        self.setLayout(layout)

        # Connect logic:
        self.model = model
        model.data_changed.connect(self.update)
        self.button.clicked.connect(model.increment)
        
        self.update()  # Make sure it's up-to-date from the start.
    
    def update(self):
        self.label.setText("Value is " + str(self.model.value()))

In [7]:
qt_app = QApplication.instance()

counter = MyCounterModel()
view = MyCounterView(counter)
view.show()

qt_app.exec_()

0

Lets start adressing the strange parts:
* The signal is declared as a static variable (class variable).
* The pyqtSignal() object doesn't actually have a connect or emit method.

It's all "magic" that the initialization of QObject takes care of. You don't need know how it happens, just how to use it.
*In order to use pyqtSignal()'s the class needs to inherit from QObject.*

# Sub-models?

Many games render the whole view using a single drawing area (not unlike the CardView class illustrated last lecture), and in these cases, there is really only 1 view that displays one 1 model. The view wouldn't really need to use widgets and layouts, and there would be a lot of custom code instead.

In applications (and we can make our poker game more into an application design) when using widgets, we might have multiple players, all using the same type of "PlayerView"-widget, displayed in a VBoxLayout.
In these cases, it makes sense to think of "sub-models", which would be isolated components that describe a local section of the application state.

In [19]:
# This is a simple game. The players take turn add 1 through 3, until they reach 20. The player who gets 20 (or higher) wins.
# This simple game doesn't motivate the complex design here, but the purpose was to illustrate the local states.
class GameState(QObject):
    # We might have multiple signals! One for updates
    data_changed = pyqtSignal()
    # and one for messaging
    game_message = pyqtSignal((str,)) # (x,) is the notation for a tuple with just one value!

    def __init__(self, players):
        super().__init__()
        self.running = False
        self.players = players
        self.player_turn = -1
        self.total = 0
        
    def start(self):
        if self.running:
            self.game_message.emit("Can't start game. Game already running")
        
        self.running = True
        self.player_turn = 0
        self.total = 0
        self.players[self.player_turn].set_active(True)
        self.data_changed.emit()

    def add(self, num):
        # Called when a player adds a value (this also switches the players turn)
        self.total += num
        if self.total >= 20:
            winner = self.players[self.player_turn]
            self.game_message.emit("Player {} won!".format(winner.name))
            winner.won()
            self.total = 0

        self.players[self.player_turn].set_active(False)
        self.player_turn = (self.player_turn + 1) % len(self.players)
        self.players[self.player_turn].set_active(True)
        self.data_changed.emit()

# A simple player state. It keeps track of the score.
class PlayerState(QObject):

    data_changed = pyqtSignal()
    
    def __init__(self, name):
        super().__init__()
        self.name = name
        self.wins = 0
        self.active = False
    
    def set_active(self, active):
        self.active = active
        self.data_changed.emit()

    def won(self):
        self.wins += 1
        self.data_changed.emit()

In [20]:
class PlayerView(QGroupBox):
    def __init__(self, player, game):
        super().__init__(player.name)
        self.player = player
        
        layout = QVBoxLayout()
        self.setLayout(layout)

        self.wins = QLabel()
        layout.addWidget(self.wins)

        self.buttons = []
        for b in range(3):
            button = QPushButton("Add {}".format(b+1))
            self.buttons.append(button)
            layout.addWidget(button)
            #button.clicked.connect( lambda : game.add(b+1) )

        def add_1(): game.add(1)
        def add_2(): game.add(2)
        def add_3(): game.add(3)
        self.buttons[0].clicked.connect(add_1)
        self.buttons[1].clicked.connect(add_2)
        self.buttons[2].clicked.connect(add_3)
            
        player.data_changed.connect(self.update)
        self.update()

    def update(self):
        self.wins.setText("Wins: " + str(self.player.wins))
        for b in self.buttons:
            b.setEnabled(self.player.active)

In [23]:
class GameView(QWidget):
    def __init__(self, game):
        super().__init__()
    
        self.game = game
        
        layoutv = QVBoxLayout()
        self.setLayout(layoutv)

        self.total_label = QLabel("uninitialized")
        layoutv.addWidget(self.total_label)
        
        layouth = QHBoxLayout()
        layoutv.addLayout(layouth)
        
        self.player_views = []
        for p in game.players:
            player_view = PlayerView(p, game)
            self.player_views.append(player_view)
            layouth.addWidget(player_view)

        game.game_message.connect(self.alert_user)
        game.data_changed.connect(self.update)
        game.start() # We start as soon as we get a view!
        self.update()

    def alert_user(self, text):
        # A method like this is nice to have for showing if the game is over, 
        # or warn about faulty input.
        box = QMessageBox()
        box.setText(text)
        box.exec_()
        
    def update(self):
        self.total_label.setText("Total: " + str(self.game.total))

In [24]:
qt_app = QApplication.instance()

model = GameState([PlayerState("Lisa"), PlayerState("Pelle")])
view = GameView(model)
view.show()

qt_app.exec_()

0

The view (which includes the controller) has 3 means of communication;
* A model of which it can ask what to draw (e.g. what cards the player has, if any)
* A signal to connect the "update" method to (which may or may not send information)
* Methods to pass information back after user interaction.

As we can see, even if we have a widget that represents one player-model, it doens't mean that widget reports changes back to that player-model. 
Typical in games, we have a controlling overreaching game logic which needs to take the input.

## Tell or ask?

What is best?
* The View ask the Model for the information it needs.
* The Model sends the information.

No right answer! As long as the model logic isn't to strongly influenced by the GUI code, little can go wrong.
It's a good idea to be consistent though. You don't have to use the Model+View seperation, but it's a good idea in terms of design.